In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tdgmaes/Etiquetado.csv


In [1]:
!pip install transformers
!pip install torch

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [3]:
dataset_path = "/kaggle/input/nv-dataset/Etiquetado nv.csv"
df = pd.read_csv(dataset_path,delimiter=';')

In [4]:
import nltk
import spacy
import re
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [5]:
df.head()

,Obligacion,Fecha Gestion,Observacion Gestion,Mes,Llave,Pago
0,11906390,01/02/2023,maria jose toma mensaje.\nsandra contesta dice...,2,11906390_2,PAGÓ
1,12731600,01/02/2023,Cliente paga el 06/02 = 364.000\nCliente paga...,2,12731600_2,PAGÓ
2,12761130,01/02/2023,"saldo total $1,439,149.32/ \nMotivo de mora / ...",2,12761130_2,NO PAGÓ
3,12780130,01/02/2023,tt informa pagar la fra el dia 7 de febrero po...,2,12780130_2,NO PAGÓ
4,12780800,01/02/2023,tt informa pagar la fra para el dia 15 de febr...,2,12780800_2,PAGÓ


In [7]:
# Descargar los recursos necesarios para NLTK
nltk.download('punkt')

# Cargar el modelo de spaCy para el idioma español
spacy.cli.download("es_core_news_sm")
nlp = spacy.load("es_core_news_sm")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 38.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [8]:
# Función para eliminar caracteres especiales usando expresiones regulares
def remove_special_characters(text):
    # Eliminar caracteres que no sean letras ni espacios
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

# Tokenizador para spaCy
def tokenize_spacy(text):
    return [token.text for token in nlp.tokenizer(text)]

# Tokenizador para NLTK
def tokenize_nltk(text):
    return nltk.word_tokenize(text)


# Preprocesamiento de texto
def preprocess_text(texts, tokenizer, special_characters=True):
    # Eliminar caracteres especiales si special_characters es True
    if special_characters:
        texts = [remove_special_characters(text) for text in texts]
    tokenized_texts = [tokenizer(text.lower()) for text in texts]
    return tokenized_texts


In [24]:
textos = df['Observacion Gestion'].tolist()
texto1 = textos[3]

In [25]:
tokenized_texts = preprocess_text(texto1, tokenize_spacy, special_characters=True)
registro = ''.join(str(caracter) for sublist in tokenized_texts for caracter in (sublist if isinstance(sublist, list) else [sublist]))
registro

'tt informa pagar la fra el dia  de febrero por valor de '

In [27]:
procesado = []

for texto in textos:
    # Verificar si el elemento es un número float y convertirlo a cadena
    if isinstance(texto, float):
        texto = str(texto)
    
    # Tokenizar el texto y unir los caracteres después del procesamiento
    tokenized_texts = preprocess_text(texto, tokenize_spacy, special_characters=True)
    texto_unido = ''.join(str(caracter) for sublist in tokenized_texts for caracter in (sublist if isinstance(sublist, list) else [sublist]))
    procesado.append(texto_unido)


In [30]:
sum = 0
contador = 0
for i in procesado:
    contador += 1
    x = len(i)
    sum += x

In [31]:
sum/contador

145.49174609578563

In [32]:
df['TEXTO PROCESADO'] = procesado

In [33]:
df

,Obligacion,Fecha Gestion,Observacion Gestion,Mes,Llave,Pago,TEXTO PROCESADO
0,11906390,01/02/2023,maria jose toma mensaje.\nsandra contesta dice...,2,11906390_2,PAGÓ,maria jose toma mensaje\nsandra contesta dice ...
1,12731600,01/02/2023,Cliente paga el 06/02 = 364.000\nCliente paga...,2,12731600_2,PAGÓ,cliente paga el \ncliente paga el
2,12761130,01/02/2023,"saldo total $1,439,149.32/ \nMotivo de mora / ...",2,12761130_2,NO PAGÓ,saldo total \nmotivo de mora se recuerda con...
3,12780130,01/02/2023,tt informa pagar la fra el dia 7 de febrero po...,2,12780130_2,NO PAGÓ,tt informa pagar la fra el dia de febrero por...
4,12780800,01/02/2023,tt informa pagar la fra para el dia 15 de febr...,2,12780800_2,PAGÓ,tt informa pagar la fra para el dia de febrer...
...,...,...,...,...,...,...,...
118969,21269250,24/07/2023,"Saldo sin Debitar,dice que tenia el dinero en ...",7,21269250_7,PAGÓ,saldo sin debitardice que tenia el dinero en l...
118970,21277230,24/07/2023,"motivo de mora pago de nomina, se le indica co...",7,21277230_7,NO PAGÓ,motivo de mora pago de nomina se le indica con...
118971,21284680,24/07/2023,motivo de mora la mama se gasto un dinero de l...,7,21284680_7,PAGÓ,motivo de mora la mama se gasto un dinero de l...
118972,21288050,24/07/2023,"Descuido y/o Olvido - 462,000.00",7,21288050_7,PAGÓ,descuido yo olvido


In [35]:
label_encoder = LabelEncoder()
df['PAGO ENCODED'] = label_encoder.fit_transform(df['Pago'])
df

,Obligacion,Fecha Gestion,Observacion Gestion,Mes,Llave,Pago,TEXTO PROCESADO,PAGO ENCODED
0,11906390,01/02/2023,maria jose toma mensaje.\nsandra contesta dice...,2,11906390_2,PAGÓ,maria jose toma mensaje\nsandra contesta dice ...,1
1,12731600,01/02/2023,Cliente paga el 06/02 = 364.000\nCliente paga...,2,12731600_2,PAGÓ,cliente paga el \ncliente paga el,1
2,12761130,01/02/2023,"saldo total $1,439,149.32/ \nMotivo de mora / ...",2,12761130_2,NO PAGÓ,saldo total \nmotivo de mora se recuerda con...,0
3,12780130,01/02/2023,tt informa pagar la fra el dia 7 de febrero po...,2,12780130_2,NO PAGÓ,tt informa pagar la fra el dia de febrero por...,0
4,12780800,01/02/2023,tt informa pagar la fra para el dia 15 de febr...,2,12780800_2,PAGÓ,tt informa pagar la fra para el dia de febrer...,1
...,...,...,...,...,...,...,...,...
118969,21269250,24/07/2023,"Saldo sin Debitar,dice que tenia el dinero en ...",7,21269250_7,PAGÓ,saldo sin debitardice que tenia el dinero en l...,1
118970,21277230,24/07/2023,"motivo de mora pago de nomina, se le indica co...",7,21277230_7,NO PAGÓ,motivo de mora pago de nomina se le indica con...,0
118971,21284680,24/07/2023,motivo de mora la mama se gasto un dinero de l...,7,21284680_7,PAGÓ,motivo de mora la mama se gasto un dinero de l...,1
118972,21288050,24/07/2023,"Descuido y/o Olvido - 462,000.00",7,21288050_7,PAGÓ,descuido yo olvido,1


In [36]:
import numpy as np
df2 = pd.DataFrame({'TEXTO PROCESADO': df['TEXTO PROCESADO'], 'PAGO ENCODED': df['PAGO ENCODED']})
df2['TEXTO PROCESADO'].replace('', np.nan, inplace=True)
df2.dropna(subset=['TEXTO PROCESADO'], inplace=True)

In [37]:
df2

,TEXTO PROCESADO,PAGO ENCODED
0,maria jose toma mensaje\nsandra contesta dice ...,1
1,cliente paga el \ncliente paga el,1
2,saldo total \nmotivo de mora se recuerda con...,0
3,tt informa pagar la fra el dia de febrero por...,0
4,tt informa pagar la fra para el dia de febrer...,1
...,...,...
118969,saldo sin debitardice que tenia el dinero en l...,1
118970,motivo de mora pago de nomina se le indica con...,0
118971,motivo de mora la mama se gasto un dinero de l...,1
118972,descuido yo olvido,1


In [38]:
df2.to_csv("preprocesado.csv")